In [1]:
# %pip install tensorflow numpy pandas scikit-learn

In [2]:
# Imports

# Core imports
import tensorflow as tf
import pandas as pd
import numpy as np

# Data manipulation
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model architecture
# from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Attention
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard


2023-12-04 02:47:23.110299: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 02:47:23.158557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 02:47:23.158587: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 02:47:23.158627: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 02:47:23.168861: I tensorflow/core/platform/cpu_feature_g

In [3]:
# Enable memory growth
# For error: Fail to find the dnn implementation.

# physical_devices = tf.config.list_physical_devices('GPU')
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
from tensorflow.keras.models import Model

### Load, Process, and Split Data

In [5]:
# Load the data

file_path = 'reddit_cleansed_data.csv'
data = pd.read_csv(file_path)


In [6]:
# Pre-process data

# To reduce tokenization size, we will lowercase all tokens
data['title'] = data['title'].str.lower()
data['selftext'] = data['selftext'].str.lower()

# We will also remove quotation marks
data['title'] = data['title'].str.replace('"', '')
data['selftext'] = data['selftext'].str.replace('"', '')

# We will also add <BOS> and <EOS> tokens to the beginning and end of every sentence for inference
data['title'] = '<BOS> ' + data['title'] + ' <EOS>'
data['selftext'] = '<BOS> ' + data['selftext'] + ' <EOS>'

data

,title,selftext,score,num_comments,gilded_count,date,timestamp
0,<BOS> do not expose any part of your body to t...,<BOS> i repeat..this is not a drill.. <EOS>,65.0,5.0,0,1.428090e+09,2015-04-03 19:47:13
1,"<BOS> i sometimes remember the way he looked, ...",<BOS> i neglected to make sure he was dead. <EOS>,22.0,0.0,0,1.428235e+09,2015-04-05 11:55:10
2,<BOS> i live alone on the third floor of my ap...,<BOS> so who opens my window every night while...,35.0,3.0,0,1.428370e+09,2015-04-07 01:24:42
3,"<BOS> i heard the rain hitting my window, so i...","<BOS> my window wasn't wet, but the glass was ...",28.0,3.0,0,1.428385e+09,2015-04-07 05:40:55
4,<BOS> you know how sometimes your brain plays ...,<BOS> i caught one of those things today. <EOS>,84.0,6.0,0,1.428563e+09,2015-04-09 07:03:16
...,...,...,...,...,...,...,...
94081,<BOS> as i look thru at window i see something...,<BOS> my reflection helps me remember how well...,31.0,2.0,0,1.680377e+09,2023-04-01 19:21:54
94082,<BOS> i’ve always been passionate about conspi...,"<BOS> so when my wife had twins, i knew exactl...",27.0,8.0,0,1.680377e+09,2023-04-01 19:24:55
94083,"<BOS> you'll see me on the red carpet one day,...","<BOS> so i paid her a surprise visit, and upon...",23.0,2.0,0,1.680378e+09,2023-04-01 19:38:03
94084,<BOS> i could hear my sister screaming nearby ...,<BOS> but my heart sank when i remembered the ...,60.0,3.0,0,1.680378e+09,2023-04-01 19:41:01


In [7]:
# Split the data
train_data, test_data = train_test_split(data, test_size=0.2)


### Tokenization and Padding

In [8]:
def preprocess_sentences(tokenizer, sentences):
    sequences = tokenizer.texts_to_sequences(sentences)
    max_length = max([len(seq) for seq in sequences])
    padded = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded, max_length

In [9]:
# Tokenize data
# tokenizer = Tokenizer()
tokenizer = Tokenizer(oov_token="<OOV>", filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(data['title'] + data['selftext'])
vocab_size = len(tokenizer.word_index) + 1

In [10]:
# Pad data

train_titles, title_max_length = preprocess_sentences(tokenizer, train_data['title'])
train_texts, text_max_length = preprocess_sentences(tokenizer, train_data['selftext'])

### Model Architecture

In [11]:
# Model parameters
embedding_dim = 256
lstm_units = 256

In [12]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Attention Layer
attention_layer = Attention()
attention_result = attention_layer([decoder_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_result])

# Dense layer
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2023-12-04 02:47:33.685508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46498 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:ce:00.0, compute capability: 8.9
2023-12-04 02:47:34.526828: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


### Model Training

In [13]:
# Callbacks
checkpoint = ModelCheckpoint('model_with_tokens.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tensorboard = TensorBoard(log_dir='./logs')


In [14]:
# Prepare decoder target data
train_texts_shifted = np.hstack([train_texts[:, 1:], np.zeros((len(train_texts), 1))])
train_texts_shifted = np.expand_dims(train_texts_shifted, -1)


In [15]:
# Train the model
model.fit([train_titles, train_texts], train_texts_shifted,
          batch_size=64,
          epochs=40,
          validation_split=0.2,
          callbacks=[checkpoint, early_stopping, tensorboard])


Epoch 1/40


2023-12-04 02:47:38.348314: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA RTX 6000 Ada Generation" frequency: 2505 num_cores: 142 environment { key: "architecture" value: "8.9" } environment { key: "cuda" value: "11080" } environment { key: "cudnn" value: "8600" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 100663296 shared_memory_size_per_multiprocessor: 102400 memory_size: 48757080064 bandwidth: 960096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-12-04 02:47:40.946264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2023-12-04 02:47:42.169006: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5b144b28b0 initialized for platform CUDA (this does not guarantee that XLA will be us

941/941 [==============================] - ETA: 0s - loss: 2.5385 - accuracy: 0.6631

2023-12-04 02:48:45.180167: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA RTX 6000 Ada Generation" frequency: 2505 num_cores: 142 environment { key: "architecture" value: "8.9" } environment { key: "cuda" value: "11080" } environment { key: "cudnn" value: "8600" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 100663296 shared_memory_size_per_multiprocessor: 102400 memory_size: 48757080064 bandwidth: 960096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_m

941/941 [==============================] - 78s 75ms/step - loss: 2.5385 - accuracy: 0.6631 - val_loss: 2.2731 - val_accuracy: 0.6748
Epoch 2/40
941/941 [==============================] - 51s 54ms/step - loss: 2.2111 - accuracy: 0.6772 - val_loss: 2.1713 - val_accuracy: 0.6806
Epoch 3/40
941/941 [==============================] - 48s 51ms/step - loss: 2.1377 - accuracy: 0.6819 - val_loss: 2.1167 - val_accuracy: 0.6840
Epoch 4/40
941/941 [==============================] - 49s 52ms/step - loss: 2.0881 - accuracy: 0.6853 - val_loss: 2.0795 - val_accuracy: 0.6868
Epoch 5/40
941/941 [==============================] - 50s 53ms/step - loss: 2.0508 - accuracy: 0.6883 - val_loss: 2.0519 - val_accuracy: 0.6884
Epoch 6/40
941/941 [==============================] - 49s 52ms/step - loss: 2.0209 - accuracy: 0.6904 - val_loss: 2.0239 - val_accuracy: 0.6910
Epoch 7/40
941/941 [==============================] - 49s 52ms/step - loss: 1.9960 - accuracy: 0.6922 - val_loss: 2.0056 - val_accuracy: 0.6927
Epo

### Inference (Generate Sentences)

In [47]:
# Extract encoder and decoder models from Seq2Seq for inference

# encoder_model = Model(encoder_inputs, encoder_states)
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs] + encoder_states)


In [34]:
# Inputs for the states from the encoder to be fed into the decoder
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Set up the decoder, using `decoder_state_input_h` and `decoder_state_input_c` as initial states
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)

# Attention inference
encoder_outputs_input = Input(shape=(None, lstm_units))
attention_result_inf = attention_layer([decoder_outputs2, encoder_outputs_input])
decoder_inf_concat = Concatenate(axis=-1)([decoder_outputs2, attention_result_inf])

# Dense layer to generate probabilities
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
    [decoder_inputs, decoder_state_input_h, decoder_state_input_c, encoder_outputs_input],
    [decoder_outputs2, state_h2, state_c2])



In [35]:
print(tokenizer.word_index)

{'bos': 1, 'eos': 2, 'the': 3, 'i': 4, 'my': 5, 'to': 6, 'and': 7, 'a': 8, 'of': 9, 'in': 10, 'was': 11, 'as': 12, 'me': 13, 'that': 14, 'it': 15, 'her': 16, 'he': 17, 'for': 18, 'when': 19, 'on': 20, 'but': 21, 'with': 22, 'you': 23, 'his': 24, 'she': 25, 'at': 26, 'from': 27, 'is': 28, 'out': 29, 'they': 30, 'had': 31, 'up': 32, 'so': 33, 'be': 34, 'have': 35, 'we': 36, 'all': 37, 'after': 38, 'said': 39, 'this': 40, 'into': 41, 'just': 42, 'one': 43, 'him': 44, '”': 45, 'not': 46, 'would': 47, 'only': 48, 'were': 49, 'what': 50, 'been': 51, 'them': 52, 'your': 53, 'back': 54, 'now': 55, 'time': 56, 'always': 57, 'our': 58, 'down': 59, 'could': 60, 'their': 61, 'an': 62, 'if': 63, 'see': 64, 'by': 65, 'like': 66, 'are': 67, 'never': 68, 'told': 69, 'no': 70, 'how': 71, 'there': 72, 'before': 73, 'saw': 74, 'last': 75, 'about': 76, 'finally': 77, 'can': 78, 'eyes': 79, 'over': 80, 'know': 81, 'night': 82, 'body': 83, 'thought': 84, 'get': 85, 'off': 86, "i'm": 87, 'found': 88, 'then':

In [52]:
# Generation with sampling and temperature control (modifies probability)

def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array with a specified temperature.
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_sentence(input_text, temperature=1.0):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=title_max_length, padding='post')

    # Get the encoder outputs and states
    encoder_output, state_h, state_c = encoder_model.predict(input_seq)

    # Start with the <BOS> token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['bos']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq, state_h, state_c, encoder_output])

        # Use temperature-based sampling to choose the next word
        sampled_token_index = sample(output_tokens[0, -1, :], temperature)

        sampled_word = tokenizer.index_word.get(sampled_token_index, 'UNK')

        if sampled_word == 'eos' or len(decoded_sentence.split()) > text_max_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        state_h, state_c = h, c

    return decoded_sentence.strip()


In [85]:
# print(generate_sentence("I was horrified when I got my test back."))
# print(generate_sentence(""))

# sentence = "My parents told me to never go upstairs."
# sentence = "I was horrified when I got my test back."
sentence = "It's crawling!"
print(generate_sentence(sentence))

1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 30ms/step
so why do the rest of his house
